In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys

import requests # to get image from the web
import shutil # to save it locally

## Custom Files:
def abspath(relative_path):
    ABS_PATH = "/home/jx/JXProject/Github/UW__4B_Individual_Works/CS 480/Kaggle"
    return os.path.join(ABS_PATH, relative_path)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(abspath("src_code"))

# Custom Lib
import jx_lib
from jx_pytorch_lib import ProgressReport


In [6]:

# %% LOAD DATASET: ----- ----- ----- ----- ----- ----- ----- ----- ----- ----- #
# import data
P_TRAIN_DATA_X = pd.read_csv(abspath("processed_data/train-[test-epoch-10-v2].csv"))
P_TEST_DATA_X = pd.read_csv(abspath("processed_data/test-[test-epoch-10-v2-test].csv"))


In [41]:
def downloadImages(tag, pd_data, n_limit=None):
    OUT_FOLDER = abspath("data/{}_img").format(tag)
    jx_lib.create_folder(OUT_FOLDER)
    total_file_size = 0
    n_retrieve = 0
    n_skipped = 0
    n_cant_retrieve = 0
    n = len(pd_data["id"])
    ni = 0
    for id, img_url in zip(pd_data["id"], pd_data["thumbnail"]):
        ni+=1
        if n_limit is not None and id > n_limit: # TEST: first 10
            break
        if pd.isna(img_url):
            pass
        else:
            file_name = os.path.join(OUT_FOLDER, "{}.png".format(id))
            file_size = 0
            if os.path.exists(file_name):
                n_skipped += 1
                file_size = os.path.getsize(file_name)/1024/1024
            else:
                # download image
                r = requests.get(img_url, stream = True)
                # Check if the image was retrieved successfully
                if r.status_code == 200:
                    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
                    r.raw.decode_content = True
                    # Open a local file with wb ( write binary ) permission.
                    with open(file_name,'wb') as f:
                        shutil.copyfileobj(r.raw, f)
                    n_retrieve += 1
                    file_size = os.path.getsize(file_name)/1024/1024
                else:
                    n_cant_retrieve += 1
            total_file_size += file_size
        print("[DOWNLOADING ... ] {}/{} total: {:.3f}MB".format(ni, n, total_file_size))
    n_actual = n_skipped + n_retrieve 
    n_tot = n_retrieve + n_cant_retrieve + n_skipped
    print("Total Image sucessfully Downloaded: {}/{}/{} : {:.2f}%/{:.2f}% | {:.3f} MB"\
        .format(n_actual, n_tot, n, n_actual/n_tot*100, n_tot/n*100, total_file_size))


In [40]:
downloadImages("train", P_TRAIN_DATA_X)


KeyboardInterrupt: 

1